In [1]:
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Nvidia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Apple
device = torch.device('mps:0' if torch.backends.mps.is_available()
                      else 'cpu')


# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread('./data'+img_path[1:])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_path = self.data.iloc[idx, 2]
        mask = cv2.imread('./data'+mask_path[1:], cv2.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12  # 배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask


# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )


# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 13, 1) # 12개 class + 1 background

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out


if __name__ == '__main__':

    transform = A.Compose(
        [
            A.Resize(224, 224),
            A.Normalize(),
            ToTensorV2()
        ]
    )

    # 데이터셋 및 DataLoader 생성
    train_csv = './data/train_source.csv'
    val_csv = './data/val_source.csv'

    train_dataset = CustomDataset(csv_file=train_csv, transform=transform)
    val_dataset = CustomDataset(csv_file=val_csv, transform=transform)

    combined_dataset = ConcatDataset([train_dataset, val_dataset])

    train_dataset, val_dataset = train_test_split(combined_dataset, test_size=0.2, random_state=42)

    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

    # model 초기화
    model = UNet().to(device)

    # loss function과 optimizer 정의
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_val_loss = np.inf
    patience = 5
    counter = 0

    # training loop
    for epoch in range(20):  # 20 에폭 동안 학습합니다.
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)

            # 추가: outputs와 masks의 차원과 크기 출력
            print(f'outputs shape: {outputs.shape}, masks shape: {masks.shape}')

            loss = criterion(outputs, masks.squeeze(1)) #???
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
          for val_images, val_masks in val_dataloader:
              val_images = val_images.float().to(device)
              val_masks = val_masks.long().to(device)

              val_outputs = model(val_images)

              val_loss += criterion(val_outputs, val_masks.squeeze(1)).item()

        val_loss /= len(val_dataloader)

        print(f'Epoch {epoch + 1}, Train Loss: {epoch_loss / len(train_dataloader)}, Val Loss: {val_loss}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0

            # 모델 가중치 저장
            torch.save(model.state_dict(), './models/trained_model_weights.pth')

            # 옵티마이저 상태 저장
            torch.save(optimizer.state_dict(), './models/optimizer_state.pth')

        else:
            counter += 1

        if counter >= patience:
            print(f'Early stopping after {epoch + 1} epochs without improvement.')
            break


  0%|                                                   | 0/133 [00:00<?, ?it/s]

outputs shape: torch.Size([16, 13, 224, 224]), masks shape: torch.Size([16, 224, 224])


  1%|▎                                          | 1/133 [00:06<15:20,  6.98s/it]

outputs shape: torch.Size([16, 13, 224, 224]), masks shape: torch.Size([16, 224, 224])


  2%|▋                                          | 2/133 [00:09<09:04,  4.16s/it]

...

outputs shape: torch.Size([16, 13, 224, 224]), masks shape: torch.Size([16, 224, 224])


100%|█████████████████████████████████████████| 133/133 [05:27<00:00,  2.46s/it]


Epoch 20, Train Loss: 0.15319125432717173, Val Loss: 0.21390103329630458


  0%|                                                   | 0/119 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File 

RuntimeError: DataLoader worker (pid(s) 44866) exited unexpectedly

In [25]:
model = UNet().to(device)

# 저장된 모델 가중치 로드
model.load_state_dict(torch.load('./models/trained_model_weights.pth'))

# 옵티마이저를 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 저장된 옵티마이저 상태 로드
optimizer.load_state_dict(torch.load('./models/optimizer_state.pth'))


test_dataset = CustomDataset(csv_file='./data/test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        outputs = model(images)
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.uint8)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

submit = pd.read_csv('./submissions/sample_submission.csv')
submit['mask_rle'] = result

submit.to_csv('./submissions/baseline_submit.csv', index=False)


100%|█████████████████████████████████████████| 119/119 [03:01<00:00,  1.53s/it]
